In [4]:
import dash
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv("titanic.csv", encoding = "latin1")

In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px

# Function to load Titanic dataset
def load_titanic_data():
    return pd.read_csv("titanic.csv")  # Replace with the actual path

# Function to create 2D survival count plot
def create_survival_count_plot(data):
    fig = px.bar(data, x='Survived', title='Survival Count')
    return fig

# Function to create 2D class distribution plot
def create_class_distribution_plot(data):
    fig = px.pie(data, names='Pclass', title='Class Distribution')
    return fig

# Function to create 3D scatter plot
def create_3d_scatter_plot(data):
    fig = px.scatter_3d(data, x='Age', y='Fare', z='Pclass', color='Survived',
                        title='3D Scatter Plot', labels={'Age': 'Age', 'Fare': 'Fare', 'Pclass': 'Class'})
    return fig

# Function to create interactive feature plot
def create_feature_plot(selected_feature, data):
    fig = px.histogram(data, x=selected_feature, color='Survived', barmode='overlay',
                       title=f'{selected_feature} Distribution by Survival',
                       labels={selected_feature: selected_feature})
    return fig

# Load the Titanic dataset
titanic_data = load_titanic_data()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Titanic Dataset Analysis Dashboard"),

    # 2D Visualizations
    html.Div([
        html.H2("Survival Count"),
        dcc.Graph(id='survival-count-plot')
    ]),

    html.Div([
        html.H2("Class Distribution"),
        dcc.Graph(id='class-distribution-plot')
    ]),

    # 3D Visualization
    html.Div([
        html.H2("3D Scatter Plot"),
        dcc.Graph(id='3d-scatter-plot')
    ]),

    # Interactive Components
    html.Div([
        html.H2("Interactive Components"),
        html.Label("Select a feature to plot against survival:"),
        dcc.Dropdown(
            id='feature-dropdown',
            options=[
                {'label': 'Age', 'value': 'Age'},
                {'label': 'Fare', 'value': 'Fare'},
                {'label': 'Pclass', 'value': 'Pclass'}
            ],
            value='Age'
        ),
        dcc.Graph(id='feature-plot')
    ])
])

# Callback to update survival count plot
@app.callback(
    Output('survival-count-plot', 'figure'),
    [Input('survival-count-plot', 'id')]
)
def update_survival_count_plot(id):
    return create_survival_count_plot(titanic_data)

# Callback to update class distribution plot
@app.callback(
    Output('class-distribution-plot', 'figure'),
    [Input('class-distribution-plot', 'id')]
)
def update_class_distribution_plot(id):
    return create_class_distribution_plot(titanic_data)

# Callback to update 3D scatter plot
@app.callback(
    Output('3d-scatter-plot', 'figure'),
    [Input('3d-scatter-plot', 'id')]
)
def update_3d_scatter_plot(id):
    return create_3d_scatter_plot(titanic_data)

# Callback to update feature plot based on user input
@app.callback(
    Output('feature-plot', 'figure'),
    [Input('feature-dropdown', 'value')],
    [State('feature-plot', 'id')]
)
def update_feature_plot(selected_feature, id):
    if selected_feature is None:
        raise dash.exceptions.PreventUpdate

    return create_feature_plot(selected_feature, titanic_data)

# Run the app
app.run_server(mode='inline')

In [11]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd

# Load the Titanic dataset
titanic_data = pd.read_csv('titanic.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    # Dropdown to select a feature
    dcc.Dropdown(
        id='feature-dropdown',
        options=[{'label': col, 'value': col} for col in titanic_data.columns],
        value='Age',  # default feature
        style={'width': '50%'}
    ),

    # Visualization components
    dcc.Graph(id='feature-plot'),
    dcc.Graph(id='survival-count-plot'),
    dcc.Graph(id='class-distribution-plot'),
    dcc.Graph(id='3d-scatter-plot'),
    dcc.Graph(id='survival-heatmap'),

    # Hidden div to store the data
    html.Div(id='titanic-data-store', style={'display': 'none'}),

    # Hidden div to store the selected feature
    html.Div(id='selected-feature-store', style={'display': 'none'})
])

# Callback to update the data store
@app.callback(
    Output('titanic-data-store', 'children'),
    [Input('feature-dropdown', 'value')]
)
def update_data_store(selected_feature):
    return titanic_data.to_json()

# Callback to update feature plot
@app.callback(
    Output('feature-plot', 'figure'),
    [Input('feature-dropdown', 'value')],
    [State('titanic-data-store', 'children')]
)
def update_feature_plot(selected_feature, titanic_data_json):
    if titanic_data_json is None:
        # Handle the case when data is not yet available
        return px.scatter(title='Waiting for data...', labels={'x': selected_feature})

    data = pd.read_json(titanic_data_json)
    return create_feature_plot(selected_feature, data)

def create_feature_plot(selected_feature, data):
    if selected_feature not in data.columns:
        raise ValueError(f"Selected feature '{selected_feature}' not found in the dataset.")

    if pd.api.types.is_numeric_dtype(data[selected_feature]):
        fig = px.histogram(data, x=selected_feature, color='Survived', barmode='overlay',
                           title=f'{selected_feature} Distribution by Survival')
    else:
        fig = px.bar(data, x=selected_feature, color='Survived',
                     title=f'{selected_feature} Count by Survival')

    return fig

# Callback to update survival count plot
@app.callback(
    Output('survival-count-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_survival_count_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.bar(title='Waiting for data...', labels={'x': 'Survived'})

    data = pd.read_json(titanic_data_json)
    return create_survival_count_plot(data)

def create_survival_count_plot(data):
    fig = px.bar(data, x='Survived', title='Survival Count')
    return fig

# Callback to update class distribution plot
@app.callback(
    Output('class-distribution-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_class_distribution_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.pie(title='Waiting for data...', labels={'names': 'Pclass'})

    data = pd.read_json(titanic_data_json)
    return create_class_distribution_plot(data)

def create_class_distribution_plot(data):
    fig = px.pie(data, names='Pclass', title='Class Distribution')
    return fig

# Callback to update 3D scatter plot
@app.callback(
    Output('3d-scatter-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_3d_scatter_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.scatter_3d(title='Waiting for data...', labels={'x': 'Age', 'y': 'Fare', 'z': 'Pclass'})

    data = pd.read_json(titanic_data_json)
    return create_3d_scatter_plot(data)

def create_3d_scatter_plot(data):
    fig = px.scatter_3d(data, x='Age', y='Fare', z='Pclass', color='Survived',
                        title='3D Scatter Plot (Age, Fare, Pclass) by Survival')
    return fig

# Callback to update survival heatmap
@app.callback(
    Output('survival-heatmap', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_survival_heatmap(titanic_data_json):
    if titanic_data_json is None:
        return px.imshow(title='Waiting for data...', labels=dict(color="Survival Correlation"))

    data = pd.read_json(titanic_data_json)
    return create_survival_heatmap(data)

def create_survival_heatmap(data):
    fig = px.imshow(data.corr(numeric_only=True), labels=dict(color="Survival Correlation"),
                    title='Survival Correlation Heatmap')
    return fig

# Run the app
if __name__ == '__main__':
    # Use a different port, for example, 8051
    app.run_server(mode='inline', port=8051)

In [13]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd

# Load the Titanic dataset
titanic_data = pd.read_csv('titanic.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    # Dropdown to select a feature
    dcc.Dropdown(
        id='feature-dropdown',
        options=[{'label': col, 'value': col} for col in titanic_data.columns],
        value='Age',  # default feature
        style={'width': '50%'}
    ),

    # Visualization components
    dcc.Graph(id='feature-plot'),
    dcc.Graph(id='survival-count-plot'),
    dcc.Graph(id='class-distribution-plot'),
    dcc.Graph(id='3d-scatter-plot'),
    dcc.Graph(id='survival-heatmap'),

    # Hidden div to store the data
    html.Div(id='titanic-data-store', style={'display': 'none'}),

    # Hidden div to store the selected feature
    html.Div(id='selected-feature-store', style={'display': 'none'})
])

# Callback to update the data store
@app.callback(
    Output('titanic-data-store', 'children'),
    [Input('feature-dropdown', 'value')]
)
def update_data_store(selected_feature):
    return titanic_data.to_json()

# Callback to update feature plot
@app.callback(
    Output('feature-plot', 'figure'),
    [Input('feature-dropdown', 'value')],
    [State('titanic-data-store', 'children')]
)
def update_feature_plot(selected_feature, titanic_data_json):
    if titanic_data_json is None:
        # Handle the case when data is not yet available
        return px.scatter(title='Waiting for data...', labels={'x': selected_feature})

    data = pd.read_json(titanic_data_json)
    return create_feature_plot(selected_feature, data)

def create_feature_plot(selected_feature, data):
    if selected_feature not in data.columns:
        raise ValueError(f"Selected feature '{selected_feature}' not found in the dataset.")

    if pd.api.types.is_numeric_dtype(data[selected_feature]):
        fig = px.histogram(data, x=selected_feature, color='Survived', barmode='overlay',
                           title=f'{selected_feature} Distribution by Survival')
    else:
        fig = px.bar(data, x=selected_feature, color='Survived',
                     title=f'{selected_feature} Count by Survival')

    return fig

# Callback to update survival count plot
@app.callback(
    Output('survival-count-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_survival_count_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.bar(title='Waiting for data...', labels={'x': 'Survived'})

    data = pd.read_json(titanic_data_json)
    return create_survival_count_plot(data)

def create_survival_count_plot(data):
    fig = px.bar(data, x='Survived', title='Survival Count')
    return fig

# Callback to update class distribution plot
@app.callback(
    Output('class-distribution-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_class_distribution_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.pie(title='Waiting for data...', labels={'names': 'Pclass'})

    data = pd.read_json(titanic_data_json)
    return create_class_distribution_plot(data)

def create_class_distribution_plot(data):
    fig = px.pie(data, names='Pclass', title='Class Distribution')
    return fig

# Callback to update 3D scatter plot
@app.callback(
    Output('3d-scatter-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_3d_scatter_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.scatter_3d(title='Waiting for data...', labels={'x': 'Age', 'y': 'Fare', 'z': 'Pclass'})

    data = pd.read_json(titanic_data_json)
    return create_3d_scatter_plot(data)

def create_3d_scatter_plot(data):
    fig = px.scatter_3d(data, x='Age', y='Fare', z='Pclass', color='Survived',
                        title='3D Scatter Plot (Age, Fare, Pclass) by Survival')
    return fig

# Callback to update survival heatmap
@app.callback(
    Output('survival-heatmap', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_survival_heatmap(titanic_data_json):
    if titanic_data_json is None:
        return px.imshow(title='Waiting for data...', labels=dict(color="Survival Correlation"))

    data = pd.read_json(titanic_data_json)
    return create_survival_heatmap(data)

def create_survival_heatmap(data):
    fig = px.imshow(data.corr(numeric_only=True), labels=dict(color="Survival Correlation"),
                    title='Survival Correlation Heatmap')
    return fig

# Run the app
if __name__ == '__main__':
    # Use a different port, for example, 8051
    app.run_server(mode='inline', port=8051)